In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chains import LLMChain

chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

memory = ConversationSummaryBufferMemory(
    llm=chat,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "movie": "Top Gun",
        "emoji": "🛩️👨‍✈️🔥",
    },
    {
        "movie": "The Godfather",
        "emoji": "👨‍👨‍👦🔫🍝",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "Tell me about the movie {movie}."),
        ("ai", "{emoji}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an AI that responds with exactly three emojis to describe movies."),
        example_prompt,
        ("human", "Show me the emojis for {movie}."),
    ]
)

chain = LLMChain(
    llm=chat,
    memory=memory,
    prompt=final_prompt,
)

def invoke_chain_and_save(movie_name):
    response = chain.invoke({"movie": movie_name})
    output_str = response.get('output', 'No response') if isinstance(response, dict) else str(response)
    
    memory.save_context({"input": movie_name}, {"output": output_str})
 
    print(output_str)

def first_asked_movie():
    history = memory.load_memory_variables({})
    first_movie_query = history.get('input', ['Top Gun'])[0]
    print(f"첫 번째로 질문한 영화: {first_movie_query}")

In [2]:
invoke_chain_and_save("Top Gun")

🛩️👨‍✈️🔥No response


In [3]:
invoke_chain_and_save("The Godfather")

👨‍👨‍👦🔫🍝No response


In [4]:
first_asked_movie()

첫 번째로 질문한 영화: Top Gun
